<a href="https://colab.research.google.com/github/sanrajbandre/parallel-computing.github.io/blob/parallel-computing-colab/colab/parallel_computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# DEVICE VERIFICATION

!nvidia-smi
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
#if device_name != b'Tesla K80':
if device_name != b'Tesla K80':
  raise Exception("Unfortunately this instance does not have a Tesla K80...!")
else:
  print('GPU DEVICE GETS VERIFIED...!')



Sun Jul 12 08:43:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    61W / 149W |   1380MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [56]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)

! ls -lrth 

b'Tesla K80'
total 215M
drwxr-xr-x 13 1000 1000 4.0K Jun  6 13:34 spark-3.0.0-bin-hadoop3.2
-rw-r--r--  1 root root 215M Jun  6 13:35 spark-3.0.0-bin-hadoop3.2.tgz
lrwxrwxrwx  1 root root   19 Jul 12 08:00 models -> /root/.torch/models
lrwxrwxrwx  1 root root   18 Jul 12 08:00 data -> /root/.fastai/data
-rw-r--r--  1 root root  517 Jul 12 08:57 demo.csv


In [55]:
# IMPORT 
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('demo.csv')
df

Saving demo.csv to demo.csv


,ID,PRODUCT,TYPE,UNIT,COST
0,1000001,PRODUCT_SERIES_1000001,SLIDE-PHONE,2,23149.0
1,1000002,PRODUCT_SERIES_1000002,SLIDE-PHONE,2,16849.0
2,1000003,PRODUCT_SERIES_1000003,KEYPAD-PHONE,1,44037.0
3,1000004,PRODUCT_SERIES_1000004,KEYPAD-PHONE,1,32585.0
4,1000005,PRODUCT_SERIES_1000005,KEYPAD-PHONE,1,26841.0
5,1000006,PRODUCT_SERIES_1000006,KEYPAD-PHONE,1,60725.0
6,1000007,PRODUCT_SERIES_1000007,SLIDE-PHONE,2,88327.0
7,1000008,PRODUCT_SERIES_1000008,KEYPAD-PHONE,1,90369.0
8,1000009,PRODUCT_SERIES_1000009,APPLE-PHONE,3,35244.0


In [4]:
# INSTALLATION GPU PACKAGES
! pip3 install cupy pycuda numba

  Using cached https://files.pythonhosted.org/packages/1d/6f/73b6509b2fd1068fad989a11aaa0ce01dbd6a6101b423c03ee264838ec74/cupy-7.6.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/5e/3f/5658c38579b41866ba21ee1b5020b8225cec86fe717e4b1c5c972de0a33c/pycuda-2019.1.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/6c/78/54e6146f65b8c6779634d28f46cad0ad6effe5f3d7614fe9892abd8e0d3c/pytools-2020.3.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/37/0e706200d22172eb8fa17d68a7ae22dec7631a0a92266634fb518a88a5b2/Mako-1.1.3-py2.py3-none-any.whl
  Created wheel for cupy: filename=cupy-7.6.0-cp36-cp36m-linux_x86_64.whl size=32978802 sha256=6df5c7241b700946d9ae5e0a58484c54f61c15259ee85b4105177fe4977a5e29
  Stored in directory: /root/.cache/pip/wheels/4f/33/12/9e457ba44352ffe2a614f7f465c0b149ebc61413b0

In [1]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy
a_gpu = gpuarray.to_gpu(numpy.random.randn(1,100000).astype(numpy.float64))
a_doubled = (2*a_gpu).get()

print (a_gpu)
print (a_doubled)

[[-0.17786392  0.86394992 -0.51056316 ...  0.37908167  0.12006722
   0.3041143 ]]
[[-0.35572785  1.72789984 -1.02112632 ...  0.75816335  0.24013443
   0.60822861]]


In [14]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6424002479998308
GPU (s):
0.19355033099964203
GPU speedup over CPU: 18x


In [34]:
! rm -vrf spark-3.0.0-bin-hadoop3.2.tgz* spark-3.0.0-preview2-bin-hadoop3.2.tgz
! wget https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
! ls -lrth
! tar -xvf spark-3.0.0-bin-hadoop3.2.tgz
! pip install -q findspark

removed 'spark-3.0.0-bin-hadoop3.2.tgz'
--2020-07-12 08:39:19--  https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224453229 (214M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop3.2.tgz’

spark-3.0.0-bin-had 100%[===================>] 214.05M  25.8MB/s    in 9.0s    

2020-07-12 08:39:28 (23.8 MB/s) - ‘spark-3.0.0-bin-hadoop3.2.tgz’ saved [224453229/224453229]

total 215M
-rw-r--r-- 1 root root 215M Jun  6 13:35 spark-3.0.0-bin-hadoop3.2.tgz
lrwxrwxrwx 1 root root   19 Jul 12 08:00 models -> /root/.torch/models
lrwxrwxrwx 1 root root   18 Jul 12 08:00 data -> /root/.fastai/data
spark-3.0.0-bin-hadoop3.2/
spark-3.0.0-bin-hadoop3.2/NOTICE
spark-3.0.0-bin-hadoop3.2/kubernetes/
spark-3.0.0-bin-hadoop3.2/kubernetes/tests/


In [51]:
! pwd
! ls -lrth /content/
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
! ls -lrth /content/spark-3.0.0-bin-hadoop3.2/examples

/content
total 215M
drwxr-xr-x 13 1000 1000 4.0K Jun  6 13:34 spark-3.0.0-bin-hadoop3.2
-rw-r--r--  1 root root 215M Jun  6 13:35 spark-3.0.0-bin-hadoop3.2.tgz
lrwxrwxrwx  1 root root   19 Jul 12 08:00 models -> /root/.torch/models
lrwxrwxrwx  1 root root   18 Jul 12 08:00 data -> /root/.fastai/data
total 8.0K
drwxr-xr-x 3 1000 1000 4.0K Jun  6 13:34 src
drwxr-xr-x 2 1000 1000 4.0K Jun  6 13:34 jars


In [40]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [48]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(5)


/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
|world|
|world|
+-----+
only showing top 5 rows



In [81]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("SPARK-3.0.0").getOrCreate()
sc = spark.sparkContext
df = spark.read.format("csv").load("demo.csv", header='true', inferSchema='true')
df.show(5)
df.groupBy("COST").count().show()


+-------+--------------------+------------+----+-------+
|     ID|             PRODUCT|        TYPE|UNIT|   COST|
+-------+--------------------+------------+----+-------+
|1000001|PRODUCT_SERIES_10...| SLIDE-PHONE|   2|23149.0|
|1000002|PRODUCT_SERIES_10...| SLIDE-PHONE|   2|16849.0|
|1000003|PRODUCT_SERIES_10...|KEYPAD-PHONE|   1|44037.0|
|1000004|PRODUCT_SERIES_10...|KEYPAD-PHONE|   1|32585.0|
|1000005|PRODUCT_SERIES_10...|KEYPAD-PHONE|   1|26841.0|
+-------+--------------------+------------+----+-------+
only showing top 5 rows

+-------+-----+
|   COST|count|
+-------+-----+
|32585.0|    1|
|90369.0|    1|
|60725.0|    1|
|23149.0|    1|
|26841.0|    1|
|44037.0|    1|
|35244.0|    1|
|88327.0|    1|
|16849.0|    1|
+-------+-----+

